In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用Vertex AI训练XGBoost开始

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/get_started_vertex_training_xgboost.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"><br> 在Colab中运行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fcustom%2Fget_started_vertex_training_xgboost.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/custom/get_started_vertex_training_xgboost.ipynb">
        <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br>
      在Vertex AI工作台中打开
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/custom/get_started_vertex_training_xgboost.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br>
      在GitHub上查看
    </a>
  </td>
  
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex AI培训XGBoost模型。

了解更多关于[自定义培训](https://cloud.google.com/vertex-ai/docs/training/custom-training)。

### 目标

在本教程中，您将学习如何使用Vertex AI Training来训练一个自定义的XGBoost模型。

本教程使用以下Google Cloud ML服务：

- Vertex AI Training
- Vertex AI模型资源

执行的步骤包括：

- 使用Python包进行训练。
- 在超参数调整时报告准确性。
- 使用GCSFuse将模型构件保存到Cloud Storage。
- 创建一个Vertex AI模型资源。

数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/iris)的鸢尾花数据集。该数据集不需要任何特征工程。本教程中的数据集版本存储在公共云存储桶中。训练模型可以预测三种鸢尾花物种中的一种：山鸢尾、维吉尼亚鸢尾或变色鸢尾。

成本

本教程使用 Google Cloud 的可计费组件:

* Vertex AI
* 云存储

了解 [Vertex AI定价](https://cloud.google.com/vertex-ai/pricing) 和 [云存储定价](https://cloud.google.com/storage/pricing), 并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预计使用量生成成本预估。

开始吧

### 为Python安装Vertex AI SDK和其他必需的软件包

### 为Python安装Vertex AI SDK和其他必需的软件包

In [ ]:
! pip3 install --upgrade google-cloud-aiplatform  --quiet

### 重新启动运行时 (仅限Colab)

为了使用新安装的包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️内核将重新启动。请等待它完成后再继续下一步。⚠️</b>
</div>

### 认证您的笔记本环境（仅适用于Colab）

在Google Colab上认证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置Google Cloud项目信息并初始化Python版的Vertex AI SDK

要开始使用Vertex AI，您必须拥有一个现有的Google Cloud项目并[启用Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。了解更多关于[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的信息。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储桶

创建一个存储桶，用于存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

如果您的存储桶尚不存在：运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $LOCATION $BUCKET_URI

### 初始化 Python 的 Vertex AI SDK

In [ ]:
import google.cloud.aiplatform as aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### 设置变量

接下来，设置一些在教程中使用的变量。

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器映像以及分配给虚拟机实例的GPU数量。例如，要使用一个GPU容器映像，并为每个VM分配4个NVIDIA Tesla K80 GPU，您可以指定：

     (aiplatform.gapic..AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，指定`(None, None)`以在CPU上运行容器映像。

了解更多关于[您所在区域的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*：预计2.3版本之前的TF版本在GPU支持方面会无法加载本教程中的自定义模型。这是一个已知问题，在TF 2.3中已修复。这是由生成在serving函数中的静态图操作引起的。如果您在自己的自定义模型中遇到此问题，请使用支持GPU的TF 2.3容器映像。

In [ ]:
TRAIN_GPU, TRAIN_NGPU = (None, None)

DEPLOY_GPU, DEPLOY_NGPU = (None, None)

设置预构建的容器

设置用于训练和预测的预构建的Docker容器镜像。

有关最新列表，请参阅[用于训练的预构建容器](https://cloud.google.com/ai-platform-unified/docs/training/pre-built-containers)。

有关最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
TRAIN_VERSION = "xgboost-cpu.1-1"
DEPLOY_VERSION = "xgboost-cpu.1-1"

TRAIN_IMAGE = "{}-docker.pkg.dev/vertex-ai/training/{}:latest".format(
    LOCATION.split("-")[0], TRAIN_VERSION
)
DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    LOCATION.split("-")[0], DEPLOY_VERSION
)

#### 设置机器类型

接下来，设置用于训练的机器类型。

- 设置变量`TRAIN_COMPUTE`以配置用于训练的虚拟机的计算资源。
- `机器类型`
  - `n1-standard`：每个vCPU 3.75GB内存。
  - `n1-highmem`：每个vCPU 6.5GB内存。
  - `n1-highcpu`：每个vCPU 0.9GB内存。
- `vCPUs`：数量为\[2, 4, 8, 16, 32, 64, 96\]

*注意：以下内容不受支持用于训练：*
- `standard`：2个vCPUs。
- `highcpu`：2、4和8个vCPUs。

*注意：您还可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
TRAIN_COMPUTE = "n1-standard-4"
print("Train machine type", TRAIN_COMPUTE)

## XGBoost训练介绍

一旦您训练了一个XGBoost模型，请将其保存在云存储位置，以便随后上传至Vertex AI模型资源。
XGBoost软件包不支持将模型保存到云存储位置。取而代之的是，按照以下步骤保存到云存储位置。

1. 将内存中的模型保存到本地文件系统（例如，model.bst）。
2. 使用gsutil将本地副本复制到指定的云存储位置。

*注意*：您可以对XGBoost模型进行超参数调整。

### 检查培训套餐

#### 包布局

在开始训练之前，看看如何为自定义训练任务组装一个Python包。解压缩后，包含以下目录/文件布局的包。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是将包安装到Docker镜像的操作环境中的说明。

文件`trainer/task.py`是执行自定义训练任务的Python脚本。*注意*，当在工作池规范中引用`trainer/task.py`时，目录斜杠被替换为一个点，文件后缀（`.py`）被去除：(`trainer.task`)。

#### 包装组装

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'cloudml-hypertune',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Iris tabular classification\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

### 为 Python 培训包创建任务脚本

接下来，您需要为驱动培训包的 `task.py` 脚本。一些重要的步骤包括：

- 命令行参数：
    - `model-dir`: 保存训练模型的位置。当使用 Vertex AI 自定义训练时，位置在环境变量 `AIP_MODEL_DIR` 中指定。
    - `dataset_data_url`: 要下载的训练数据的位置。
    - `dataset_labels_url`: 要下载的训练标签的位置。
    - `boost-rounds`: 可调节的超参数
- 数据预处理 (`get_data()`)：
    - 下载数据集并分割为训练和测试集。
- 训练 (`train_model()`)：
    - 训练模型。
- 评估 (`evaluate_model()`)：
    - 评估模型。
    - 如果进行超参数调优，则报告准确率指标。
- 模型工件保存
    - 将模型工件和评估指标保存到由 `model-dir` 指定的 Cloud 存储位置。

*注意:* 训练脚本使用 GCSFuse，将 Cloud 存储桶挂载为网络文件系统，使脚本能够在 Cloud 存储桶内执行文件系统操作（如读取和写入）。

In [ ]:
%%writefile custom/trainer/task.py
import datetime
import os
import subprocess
import sys
import pandas as pd
import xgboost as xgb
import hypertune
import argparse
import logging
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument("--dataset-data-url", dest="dataset_data_url",
                    type=str, help="Download url for the training data.")
parser.add_argument("--dataset-labels-url", dest="dataset_labels_url",
                    type=str, help="Download url for the training data labels.")
parser.add_argument("--boost-rounds", dest="boost_rounds",
                    default=20, type=int, help="Number of boosted rounds") 
args = parser.parse_args()

logging.getLogger().setLevel(logging.INFO)

def get_data():
    logging.info("Downloading training data and labelsfrom: {}, {}".format(args.dataset_data_url, args.dataset_labels_url))
    # gsutil outputs everything to stderr so we need to divert it to stdout.
    subprocess.check_call(['gsutil', 'cp', args.dataset_data_url, 'data.csv'], stderr=sys.stdout)
    # gsutil outputs everything to stderr so we need to divert it to stdout.
    subprocess.check_call(['gsutil', 'cp', args.dataset_labels_url, 'labels.csv'], stderr=sys.stdout)


    # Load data into pandas, then use `.values` to get NumPy arrays
    data = pd.read_csv('data.csv').values
    labels = pd.read_csv('labels.csv').values

    # Convert one-column 2D array into 1D array for use with XGBoost
    labels = labels.reshape((labels.size,))

    train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=7)

    # Load data into DMatrix object
    dtrain = xgb.DMatrix(train_data, label=train_labels)
    return dtrain, test_data, test_labels

def train_model(dtrain):
    logging.info("Start training ...")
    # Train XGBoost model
    params = {
        'objective': 'multi:softprob',
        'num_class': 3
    }
    model = xgb.train(params, dtrain, num_boost_round=args.boost_rounds)
    logging.info("Training completed")
    return model

def evaluate_model(model, test_data, test_labels):
    dtest = xgb.DMatrix(test_data)
    pred = model.predict(dtest)
    predictions = [np.around(value) for value in pred]
    # evaluate predictions
    try:
        accuracy = accuracy_score(test_labels, predictions)
    except:
        accuracy = 0.0
    logging.info(f"Evaluation completed with model accuracy: {accuracy}")

    # report metric for hyperparameter tuning
    hpt = hypertune.HyperTune()
    hpt.report_hyperparameter_tuning_metric(
        hyperparameter_metric_tag='accuracy',
        metric_value=accuracy
    )
    return accuracy


dtrain, test_data, test_labels = get_data()
model = train_model(dtrain)
accuracy = evaluate_model(model, test_data, test_labels)

# GCSFuse conversion
gs_prefix = 'gs://'
gcsfuse_prefix = '/gcs/'
if args.model_dir.startswith(gs_prefix):
    args.model_dir = args.model_dir.replace(gs_prefix, gcsfuse_prefix)
    dirpath = os.path.split(args.model_dir)[0]
    if not os.path.isdir(dirpath):
        os.makedirs(dirpath)

# Export the classifier to a file
gcs_model_path = os.path.join(args.model_dir, 'model.bst')
logging.info("Saving model artifacts to {}". format(gcs_model_path))
model.save_model(gcs_model_path)

logging.info("Saving metrics to {}/metrics.json". format(args.model_dir))
gcs_metrics_path = os.path.join(args.model_dir, 'metrics.json')
with open(gcs_metrics_path, "w") as f:
    f.write(f"{'accuracy: {accuracy}'}")

请将培训脚本存储在您的云存储桶中。

接下来，您将培训文件夹打包成压缩的 tar 文件，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_URI/trainer_iris.tar.gz

### 创建并运行自定义训练任务

要训练一个自定义模型，您需要执行两个步骤：1）创建一个自定义训练任务，2）运行该任务。

#### 创建自定义训练任务

使用`CustomTrainingJob`类创建一个自定义训练任务，包括以下参数：

- `display_name`：自定义训练任务的可读名称。
- `container_uri`：训练容器镜像。
- `python_package_gcs_uri`：Python训练包的位置，作为一个tarball。
- `python_module_name`：Python包中训练脚本的相对路径。
- `model_serving_container_uri`：用于部署模型的容器镜像。

*注意*：没有requirements参数。您可以在Python包中的`setup.py`脚本中指定任何要求。

In [ ]:
DISPLAY_NAME = "iris"

job = aiplatform.CustomPythonPackageTrainingJob(
    display_name=DISPLAY_NAME,
    python_package_gcs_uri=f"{BUCKET_URI}/trainer_iris.tar.gz",
    python_module_name="trainer.task",
    container_uri=TRAIN_IMAGE,
    model_serving_container_image_uri=DEPLOY_IMAGE,
    project=PROJECT_ID,
)

### 准备您的命令行参数

现在为您的自定义训练容器定义命令行参数：

- `args`：传递给作为容器入口点设置的可执行文件的命令行参数。
  - `--model-dir`：对于我们的演示，我们使用此命令行参数来指定存储模型工件的位置。
      - 直接：将Cloud存储位置作为命令行参数传递给训练脚本（设置变量`DIRECT = True`），或
      - 间接：服务将Cloud存储位置作为环境变量`AIP_MODEL_DIR`传递给训练脚本（设置变量`DIRECT = False`）。在这种情况下，您需要在作业规范中告诉服务模型工件位置。
  - `--dataset-data-url`：要下载的训练数据的位置。
  - `--dataset-labels-url`：要下载的训练标签的位置。
  - `--boost-rounds`：可调整的超参数。

In [ ]:
MODEL_DIR = "{}/{}".format(BUCKET_URI, "model")
DATASET_DIR = "gs://cloud-samples-data/ai-platform/iris"

ROUNDS = 20

DIRECT = False
if DIRECT:
    CMDARGS = [
        "--dataset-data-url=" + DATASET_DIR + "/iris_data.csv",
        "--dataset-labels-url=" + DATASET_DIR + "/iris_target.csv",
        "--boost-rounds=" + str(ROUNDS),
        "--model_dir=" + MODEL_DIR,
    ]
else:
    CMDARGS = [
        "--dataset-data-url=" + DATASET_DIR + "/iris_data.csv",
        "--dataset-labels-url=" + DATASET_DIR + "/iris_target.csv",
        "--boost-rounds=" + str(ROUNDS),
    ]

运行自定义训练作业

接下来，您可以通过调用`run`方法并传入以下参数来启动训练作业：

- `model_display_name`：`Model`资源的可读名称。
- `args`：传递给训练脚本的命令行参数。
- `replica_count`：用于训练的计算实例数量（replica_count = 1表示单节点训练）。
- `machine_type`：计算实例的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要连接到工作人员副本的加速器数量。
- `base_output_dir`：用于将模型工件写入的Cloud Storage位置。
- `sync`：是否阻塞直到作业完成。

In [ ]:
if TRAIN_GPU:
    model = job.run(
        model_display_name="iris",
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        accelerator_type=TRAIN_GPU.name,
        accelerator_count=TRAIN_NGPU,
        base_output_dir=MODEL_DIR,
        sync=False,
    )
else:
    model = job.run(
        model_display_name="iris",
        args=CMDARGS,
        replica_count=1,
        machine_type=TRAIN_COMPUTE,
        base_output_dir=MODEL_DIR,
        sync=False,
    )

model_path_to_deploy = MODEL_DIR

### 列出一个定制的培训工作

In [ ]:
_job = job.list(filter=f"display_name={DISPLAY_NAME}")
print(_job)

### 等待定制培训任务完成

接下来，等待定制培训任务完成。或者可以在`run()`方法中将参数`sync`设置为`True`，以阻塞直到定制培训任务完成。

In [ ]:
model.wait()

删除自定义训练作业

训练作业完成后，您可以使用 `delete()` 方法来删除训练作业。在完成之前，可以使用 `cancel()` 方法取消训练作业。

In [ ]:
job.delete()

清理工作
要清理此项目中使用的所有Google Cloud资源，您可以[删除用于本教程的Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- Custom Job（自定义培训作业在前一步中已删除）
- 模型
- 云存储桶

In [ ]:
delete_bucket = False

model.delete()

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI